<a href="https://colab.research.google.com/github/karmanandan/gif_search_with_pinecone_vectordb/blob/main/gif_search_pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget https://github.com/raingo/TGIF-Release/archive/master.zip

--2023-08-09 07:54:12--  https://github.com/raingo/TGIF-Release/archive/master.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/raingo/TGIF-Release/zip/refs/heads/master [following]
--2023-08-09 07:54:13--  https://codeload.github.com/raingo/TGIF-Release/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 20.27.177.114
Connecting to codeload.github.com (codeload.github.com)|20.27.177.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [     <=>            ]  11.82M  6.10MB/s    in 1.9s    

2023-08-09 07:54:15 (6.10 MB/s) - ‘master.zip’ saved [12396861]



In [6]:
# !unzip /content/master.zip -d /content/target_GIF_zip

In [7]:
import pandas as pd

# Load dataset to a pandas dataframe
df = pd.read_csv(
    "/content/target_GIF_zip/TGIF-Release-master/data/tgif-v1.0.tsv",
    delimiter="\t",
    names=['url', 'description']
)
df.head()

,url,description
0,https://38.media.tumblr.com/9f6c25cc350f12aa74...,"a man is glaring, and someone with sunglasses ..."
1,https://38.media.tumblr.com/9ead028ef62004ef6a...,a cat tries to catch a mouse on a tablet
2,https://38.media.tumblr.com/9f43dc410be85b1159...,a man dressed in red is dancing.
3,https://38.media.tumblr.com/9f659499c8754e40cf...,an animal comes close to another in the jungle
4,https://38.media.tumblr.com/9ed1c99afa7d714118...,a man in a hat adjusts his tie and makes a wei...


In [21]:
from IPython.display import Image, HTML,display
for _, gif in df[:5].iterrows():
  display(HTML(f"<img src={gif['url']} style='width:120px; height:90px'>"))
  print(gif["description"])

a man is glaring, and someone with sunglasses appears.


a cat tries to catch a mouse on a tablet


a man dressed in red is dancing.


an animal comes close to another in the jungle


a man in a hat adjusts his tie and makes a weird face.


In [23]:
dupes = df['url'].value_counts().sort_values(ascending=False)
dupes.head(10)

https://38.media.tumblr.com/ddbfe51aff57fd8446f49546bc027bd7/tumblr_nowv0v6oWj1uwbrato1_500.gif    4
https://33.media.tumblr.com/46c873a60bb8bd97bdc253b826d1d7a1/tumblr_nh7vnlXEvL1u6fg3no1_500.gif    4
https://38.media.tumblr.com/b544f3c87cbf26462dc267740bb1c842/tumblr_n98uooxl0K1thiyb6o1_250.gif    4
https://33.media.tumblr.com/88235b43b48e9823eeb3e7890f3d46ef/tumblr_nkg5leY4e21sof15vo1_500.gif    4
https://31.media.tumblr.com/69bca8520e1f03b4148dde2ac78469ec/tumblr_npvi0kW4OD1urqm0mo1_400.gif    4
https://38.media.tumblr.com/95eebb6c0fd5499017d9825d52a0dbfa/tumblr_noka4jzEvu1uvvbrro1_250.gif    4
https://31.media.tumblr.com/36b594b9b404403f652e5f7b1cf65cf4/tumblr_na5aodi5wy1tph724o1_400.gif    4
https://31.media.tumblr.com/3260c7927d82fe0ea923206e4a0202a7/tumblr_nqw325qgEL1uyrkl0o1_400.gif    4
https://33.media.tumblr.com/9e92e1f4cd38366881d8c6db2af7f01d/tumblr_ne1ho9oVR41slj978o1_400.gif    4
https://38.media.tumblr.com/d75cf12fbd1474e5ff6c4daed728217d/tumblr_nplgqa6TnF1su25v0o1_400

In [25]:
def dup_images(dupe_url):
  dupe_df = df[df['url'] == dupe_url]

  # let's take a look at this GIF and it's duplicated descriptions
  for _, gif in dupe_df.iterrows():
      display(HTML(f"<img src={gif['url']} style='width:120px; height:90px'>"))
      print(gif["description"])

dup_images(dupe_url = "https://33.media.tumblr.com/88235b43b48e9823eeb3e7890f3d46ef/tumblr_nkg5leY4e21sof15vo1_500.gif")

two girls are singing music pop in a concert


a woman sings sang girl on a stage singing


two girls on a stage sing into microphones.


two girls dressed in black are singing.


In [26]:
dup_images(dupe_url = "https://38.media.tumblr.com/d75cf12fbd1474e5ff6c4daed728217d/tumblr_nplgqa6TnF1su25v0o1_400.gif")

a child is petting a sheep in the street


a baby wears a white coat and plays with a sheep


a baby in a white coat is petting a lamb.


a young child is playing with a baby sheep on a leash


In [28]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.9 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=98fa6cc1dda36fa9c082fc59318589e8ac48cb379848c05ca4bd3920053cfe76
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [29]:
from sentence_transformers import SentenceTransformer

# Initialize retriever with SentenceTransformer model
retriever = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
retriever

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [32]:
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 7.4 MB/s eta 0:00:00


In [36]:
import pinecone

# Connect to pinecone environment
pinecone.init(
    api_key="",
    environment="us-west1-gcp-free"  # find next to API key
)

index_name = 'gif-search'

# check if the gif-search exists
if index_name not in pinecone.list_indexes():
    # create the index if it does not exist
    pinecone.create_index(
        index_name,
        dimension=384,
        metric="cosine"
    )

# Connect to gif-search index we created
index = pinecone.Index(index_name)

In [38]:
from tqdm.auto import tqdm

# we will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(df), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(df))
    # extract batch
    batch = df.iloc[i:i_end]
    # generate embeddings for batch
    emb = retriever.encode(batch['description'].tolist()).tolist()
    # get metadata
    meta = batch.to_dict(orient='records')
    # create IDs
    ids = [f"{idx}" for idx in range(i, i_end)]
    # add all to upsert list
    to_upsert = list(zip(ids, emb, meta))
    # upsert/insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)


# check that we have all vectors in index
index.describe_index_stats()

  0%|          | 0/1966 [00:00<?, ?it/s]

{'dimension': 384,
 'index_fullness': 0.4,
 'namespaces': {'': {'vector_count': 125782}},
 'total_vector_count': 125782}

In [40]:
def search_gif(query):
    # Generate embeddings for the query
    xq = retriever.encode(query).tolist()
    # Compute cosine similarity between query and embeddings vectors and return top 10 URls
    xc = index.query(xq, top_k=10,
                    include_metadata=True)
    result = []
    for context in xc['matches']:
        url = context['metadata']['url']
        result.append(url)
    return result

In [41]:
def display_gif(urls):
    figures = []
    for url in urls:
        figures.append(f'''
            <figure style="margin: 5px !important;">
              <img src="{url}" style="width: 120px; height: 90px" >
            </figure>
        ''')
    return display(HTML(data=f'''
        <div style="display: flex; flex-flow: row wrap; text-align: center;">
        {''.join(figures)}
        </div>
    '''))

In [42]:
gifs = search_gif("a dog being confused")
display_gif(gifs)

In [43]:
gifs = search_gif("a boy looks angry")
display_gif(gifs)

In [45]:
gifs = search_gif("eating food")
display_gif(gifs)

In [ ]:
# pinecone.delete_index(index_name)